### Importación de Bibliotecas

In [1]:
import pandas as pd
import os

### Cargar el conjunto de datos limpios

In [2]:
df_steam_games = pd.read_csv('steam_games_cleaned.csv')
df_user_reviews = pd.read_csv('user_reviews_cleaned.csv')

### Rutinas

##### - 3. UsersRecommend

In [3]:
# Merge de los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
result = df_merged.loc[(df_merged['recommend'] == True) & (df_merged['sentiment_analysis'].isin([1, 2])),
                           ['year_x', 'name']]

# Renombrar la columna 'year_x' a 'year'
result = result.rename(columns={'year_x': 'year'})

# Agrupar por 'year' y 'name' y contar las ocurrencias
grouped_result = result.groupby(['year', 'name']).size().reset_index(name='count')

# Ordenar por 'year' y 'count' en orden descendente
grouped_result = grouped_result.sort_values(by=['year', 'count'], ascending=[False, False])

# Obtener el top 3 por año
top3_by_year = grouped_result.groupby('year').head(3)

# Guardar el resultado en un archivo CSV
top3_by_year.to_csv('UsersRecommend.csv', index=False)

##### - 4. UsersWorstDeveloper

In [9]:
# Unir los DataFrames
df_merged = pd.merge(df_user_reviews, df_steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
df_filtered = df_merged.loc[(df_merged['recommend'] == False) & (df_merged['sentiment_analysis'] == 0),
                           ['year_x', 'developer']]
 
# Renombrar la columna 'year_x' a 'year'
df_filtered = df_filtered.rename(columns={'year_x': 'year'})

# Contar las ocurrencias de cada desarrolladora por año
developer_counts = df_filtered.groupby(['year', 'developer']).size().reset_index(name='count')
    
# Ordenar por 'year' y 'count' en orden descendente
grouped_result = developer_counts.sort_values(by=['year', 'count'], ascending=[False, False])    

# Obtener el top 3 por año
result_df = grouped_result.groupby('year').head(3)    

# Guardar el resultado en un archivo CSV
result_df.to_csv('UsersWorstDeveloper.csv', index=False)

##### - 5. sentiment_analysis

In [8]:
# Merge de los DataFrames
merged_df = pd.merge(df_steam_games, df_user_reviews, on='item_id', how='inner')

# Agrupar por desarrolladora y sentimiento, y contar las ocurrencias
result_df = merged_df.groupby(['developer', 'sentiment_analysis']).size().reset_index(name='count')

# Pivoteando el DataFrame para tener sentimientos como columnas
result_df = result_df.pivot(index='developer', columns='sentiment_analysis', values='count').reset_index()

# Renombrar las columnas según el formato deseado
result_df.columns = ['developer', 'Negative', 'Neutral', 'Positive']

# Rellenar NaN con 0
result_df = result_df.fillna(0)

# Convertir las columnas a tipo int
result_df[['Negative', 'Neutral', 'Positive']] = result_df[['Negative', 'Neutral', 'Positive']].astype(int)

# Guardar el resultado en un archivo CSV
result_df.to_csv('sentiment_analysis.csv', index=False)